# <font size="5">Libraries</font>

In [3]:
%load_ext autoreload
%autoreload 2

import logging
import torch
import pytorch_lightning as pl
import warnings

warnings.filterwarnings('ignore')
logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'pytorch_lightning'

In [4]:
from glob import glob
import numpy as np
import logging
import pytorch_lightning as pl
import torch
from tqdm.auto import tqdm
import os
import pandas as pd
from pytorch_lightning.loggers import TensorBoardLogger


logger = logging.getLogger(__name__)

ModuleNotFoundError: No module named 'pytorch_lightning'

# <font size="5">Load model</font>

In [3]:
import hydra
from omegaconf import OmegaConf

conf = OmegaConf.load('mles_params.yaml')
model = hydra.utils.instantiate(conf.pl_module)
model.load_state_dict(torch.load("models_alpha_battle/coles_pretrain_without_last.pth"))

<All keys matched successfully>

# <font size="5">data module</font>

In [4]:
df = pd.read_parquet('train_without_last.parquet')
df = df[['app_id', 'mcc', 'hour_diff', 'amnt_bins']]

min_value = df['hour_diff'].min()
max_value = df['hour_diff'].max()
df['normalized'] = (df['hour_diff'] - min_value) / (max_value - min_value)

# df['denormalized'] = df['normalized'] * (max_value - min_value) + min_value

df = df.groupby('app_id').agg(lambda x: list(x)).reset_index()
df.to_parquet('normalize_agg_without_last.parquet')

In [5]:
# df_valid = pd.read_parquet('data_2/valid.parquet')
df_valid = pd.read_parquet('data_2/valid.parquet')
df_valid = df_valid.groupby('app_id').head(1)
df_valid = df_valid[['app_id', 'mcc', 'hour_diff', 'amnt_bins']]

min_value = df_valid['hour_diff'].min()
max_value = df_valid['hour_diff'].max()
df_valid['normalized'] = (df_valid['hour_diff'] - min_value) / (max_value - min_value)

# df['denormalized'] = df['normalized'] * (max_value - min_value) + min_value

# df_valid = df_valid.groupby('app_id').agg(lambda x: list(x)).reset_index()
df_valid.to_parquet('data_2/normalize_agg_valid_without_last.parquet')

In [19]:
df_valid

,app_id,mcc,hour_diff,amnt_bins,normalized
0,0,2,0,2,0.000000
19,1,2,37,3,0.051389
49,2,28,156,1,0.216667
64,4,9,73,3,0.101389
75,6,58,24,3,0.033333
...,...,...,...,...,...
11183041,1003041,3,17,0,0.023611
11183077,1003044,4,0,2,0.000000
11183112,1003047,15,42,3,0.058333
11183129,1003048,12,143,3,0.198611


In [24]:
df.merge(df_valid.drop(columns=['mcc',
                                'hour_diff',
                                'amnt_bins',
                                'normalized']), on='app_id', how='inner')

,app_id,mcc,hour_diff,amnt_bins,normalized
0,0,"[2, 2, 2, 10, 2, 2, 2, 2, 10, 10, 2, 10, 2, 2,...","[-1, 0, 0, 52, 280, 0, 75, 169, 18, 0, 146, 71...","[3, 0, 3, 3, 0, 3, 0, 0, 3, 2, 3, 3, 0, 3, 3, ...","[0.0, 0.00014423770373575654, 0.00014423770373..."
1,1,"[65, 1, 2, 2, 1, 3, 58, 2, 1, 2, 1, 1, 2, 25, ...","[-1, 24, 19, 74, 517, 34, 1, 92, 0, 22, 6, 19,...","[1, 3, 3, 3, 1, 0, 3, 3, 0, 3, 3, 0, 1, 0, 3, ...","[0.0, 0.003605942593393913, 0.0028847540747151..."
2,2,"[3, 1, 7, 1, 28, 28, 1, 2, 28, 11, 7, 7, 1, 1,...","[-1, 10, 14, 21, 2, 0, 25, 875, 57, 64, 0, 0, ...","[3, 3, 3, 3, 0, 3, 3, 3, 0, 3, 2, 0, 0, 0, 3, ...","[0.0, 0.0015866147410933218, 0.002163565556036..."
3,4,"[2, 2, 1, 1, 9, 9, 2, 9, 9, 2, 9, 9, 9, 2, 9, ...","[-1, 0, 0, 5, 1, 0, 260, 27, 0, 163, 315, 0, 1...","[0, 3, 3, 0, 3, 0, 3, 3, 0, 3, 3, 3, 3, 3, 3, ...","[0.0, 0.00014423770373575654, 0.00014423770373..."
4,6,"[1, 4, 14, 14, 14, 14, 4, 25, 1, 1, 4, 4, 1, 1...","[-1, 18, 5, 0, 23, 0, 20, 0, 0, 0, 0, 0, 22, 0...","[0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 2, 1, 1, 2, ...","[0.0, 0.002740516370979374, 0.0008654262224145..."
...,...,...,...,...,...
429706,1003041,"[2, 12, 2, 2, 11, 4, 2, 11, 9, 36, 11, 1, 12, ...","[-1, 334, 168, 30, 815, 166, 72, 457, 0, 234, ...","[3, 1, 3, 0, 3, 3, 3, 3, 3, 3, 3, 2, 3, 1, 0, ...","[0.0, 0.04831963075147844, 0.02437617193134285..."
429707,1003044,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 9, 5, ...","[-1, 22, 4, 67, 5, 1, 19, 3, 46, 3, 14, 80, 3,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, ...","[0.0, 0.0033174671859224, 0.000721188518678782..."
429708,1003047,"[4, 1, 10, 10, 4, 1, 10, 10, 105, 5, 5, 36, 1,...","[-1, 91, 20, 0, 7, 22, 145, 0, 2, 2, 7, 175, 6...","[3, 3, 3, 2, 2, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, ...","[0.0, 0.0132698687436896, 0.003028991778450887..."
429709,1003048,"[53, 18, 18, 35, 18, 9, 18, 2, 2, 12, 18, 18, ...","[-1, 7, 4, 42, 52, 40, 37, 12, 0, 5, 148, 50, ...","[3, 0, 0, 3, 2, 3, 1, 3, 3, 3, 2, 2, 3, 3, 3, ...","[0.0, 0.0011539016298860523, 0.000721188518678..."


In [27]:
df.to_parquet('df_train_agg_without_last.parquet')

In [44]:
from glob import glob

from ptls.data_load.iterable_processing_dataset import IterableProcessingDataset
from ptls.data_load.iterable_processing.target_move import TargetMove
from ptls.data_load.iterable_processing.target_empty_filter import TargetEmptyFilter
from ptls.data_load import padded_collate, padded_collate_wo_target
from ptls.data_load.iterable_processing.to_torch_tensor import ToTorch
from ptls.data_load.datasets import MemoryMapDataset
from tqdm.auto import tqdm

from ptls.data_load import IterableChain
from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.data_load.datasets.parquet_dataset import ParquetDataset, ParquetFiles
from ptls.data_load.utils import collate_feature_dict

from ptls.frames import PtlsDataModule

train_data = glob('data_2/normalize_agg.parquet')
valid_data = glob('data_2/normalize_agg_valid.parquet')

feature_cols = ['mcc', 'amnt_bins', 'hour_diff']

target_cols = ['mcc', 'amnt_bins', 'normalized']

dataset_conf = {
    'min_seq_len':0,
}

class SeqToTargetMultiheadDataset(torch.utils.data.Dataset):
    def __init__(self,
                 data,
                 feature_cols,
                 target_cols,
                 target_dtype=None,
                 *args, **kwargs,
                 ):
        super().__init__(*args, **kwargs)

        self.data = data
        self.feature_cols = feature_cols
        self.target_cols = target_cols
        
        if type(target_dtype) is str:
            self.target_dtype = getattr(torch, target_dtype)
        else:
            self.target_dtype = target_dtype

    def __len__(self):
        return len(self.data)

    def __getitem__(self, item):
        feature_arrays = self.data[item]
        return feature_arrays

    def __iter__(self):
        for feature_arrays in self.data:
            yield feature_arrays


    def collate_fn(self, batch):
        
        targets = []
        values = []
        
        for target_col in self.target_cols:
            targets.append(torch.tensor([rec[target_col][-1] for rec in batch]).to(self.target_dtype[target_col]))
        
        for rec in batch:
            values.append({k: v[:-1] for k, v in rec.items() if k in feature_cols})
    
        return padded_collate_wo_target(values), targets

process = IterableChain(
            SeqLenFilter(min_seq_len=dataset_conf['min_seq_len']),
            ToTorch()
            )
    
def get_dataset(data):
    ds = MemoryMapDataset(ParquetDataset(data, post_processing=process))
    return SeqToTargetMultiheadDataset(ds, feature_cols, target_cols, target_dtype = {'mcc': torch.long, 'amnt': torch.float, 'normalized': torch.float})

train_ds = get_dataset(train_data)
valid_ds = get_dataset(valid_data)

dm = PtlsDataModule(
    train_data=train_ds,
    train_num_workers=4,
    train_batch_size=64)

In [14]:
import logging
from copy import deepcopy
from typing import List

import pandas as pd
import pytorch_lightning as pl
import torch
import torchmetrics
from omegaconf import DictConfig

from ptls.data_load.padded_batch import PaddedBatch

logger = logging.getLogger(__name__)


class SequenceToTargetMultihead(pl.LightningModule):


    def __init__(self,
                 seq_encoder: torch.nn.Module,
                 heads: List[torch.nn.Module],
                 losses: List[torch.nn.Module],
                 metric_list: torchmetrics.Metric=None,
                 optimizer_partial=None,
                 lr_scheduler_partial=None,
                 pretrained_lr=None,
                 train_update_n_steps=None,
                 ):
        super().__init__()

        self.save_hyperparameters(ignore=[
            'seq_encoder', 'heads', 'losses', 'metric_list', 'optimizer_partial', 'lr_scheduler_partial'])

        self.seq_encoder = seq_encoder
        self.heads = heads
        self.losses = losses
        self.n_heads = len(heads)

        self.optimizer_partial = optimizer_partial
        self.lr_scheduler_partial = lr_scheduler_partial

    def forward(self, x):
        x = self.seq_encoder(x)
        xs = [head(x) for head in self.heads]
        return xs

    def training_step(self, batch, _):
        x, y = batch
        y_hs = self(x)
        loss = sum([loss(y_hs[i], y[i]) for i, loss in enumerate(self.losses)])
        self.log('loss', loss)
        return loss

    def validation_step(self, batch, _):
        x, y = batch
        y_hs = self(x)
        loss = sum([loss(y_hs[i], y[i]) for i, loss in enumerate(self.losses)])
        self.log('val_loss', loss)


    def configure_optimizers(self):
        if self.hparams.pretrained_lr is not None:
            if self.hparams.pretrained_lr == 'freeze':
                for p in self.seq_encoder.parameters():
                    p.requires_grad = False
                parameters = self.parameters()
            else:
                parameters = [
                    {'params': self.seq_encoder.parameters(), 'lr': self.hparams.pretrained_lr},
                ] + [{'params': head.parameters()} for head in self.heads]
        else:
            parameters = self.parameters()

        optimizer = self.optimizer_partial(parameters)
        scheduler = self.lr_scheduler_partial(optimizer)
        return [optimizer], [scheduler]

# <font size="5">heads</font>

In [15]:
from functools import partial
import torch
import torchmetrics
from ptls.nn import Head


head_mcc = Head(input_size=model.seq_encoder.embedding_size, 
                use_batch_norm=True,
                hidden_layers_sizes=[128],
                objective='classification',
                num_classes=109).to('cuda:0')

head_amnt = Head(input_size=model.seq_encoder.embedding_size, 
                 use_batch_norm=True,
                 hidden_layers_sizes=[128],
                 objective='softplus').to('cuda:0')

head_hour_diff = Head(input_size=model.seq_encoder.embedding_size, 
                      use_batch_norm=True,
                      hidden_layers_sizes=[128],
                      objective='softplus').to('cuda:0')

model_multihead = SequenceToTargetMultihead(
    seq_encoder=model.seq_encoder,
    heads=[head_mcc, head_amnt, head_hour_diff],
    losses=[torch.nn.NLLLoss(), torch.nn.L1Loss(), torch.nn.L1Loss()],
#     metric_list=torchmetrics.Accuracy(compute_on_step=False),
    pretrained_lr=0.00001,
    optimizer_partial=partial(torch.optim.Adam, lr=0.001), # , weight_decay=1e-5
    lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=1, gamma=0.9),
)

# <font size="5">training</font>

In [ ]:
logger = TensorBoardLogger('src/ptls-experiments/scenario_alpha_battle/lightning_logs',
                           name='coles-baseline-alpha-battle')

trainer = pl.Trainer(
    num_sanity_val_steps=0,
    gpus=1,
    auto_select_gpus=False,
    max_epochs=5,
    enable_checkpointing=False,
    deterministic=True,
    logger=logger,
)

trainer.fit(model_multihead, dm)
print(trainer.logged_metrics)

Training: 0it [00:00, ?it/s]

# <font size="5">save model</font>

In [17]:
torch.save(model_multihead.state_dict(), "models_alpha_battle/coles-multi-head_norm.pth")

# <font size="5">load model</font>

In [18]:
model_multihead.load_state_dict(torch.load("models_alpha_battle/coles-multi-head_norm.pth"))

<All keys matched successfully>

# <font size="5">Inference</font>

In [40]:
# %%time
import tqdm

def inference(model, dl, device='cuda:0'):
    
    model.to(device)
    X = []
    for batch in tqdm.tqdm(dl):
        with torch.no_grad():
            features = batch[0]
            targets = [t.unsqueeze(dim=1).to(device) for t in batch[1]]
            x = model(features.to(device))
            mcc = torch.argmax(x[0], dim=1, keepdim=True)
            amnt = x[1].unsqueeze(dim=1)
            hour_diff = x[2].unsqueeze(dim=1)
            predicted = [mcc, amnt, hour_diff]
            X += [torch.cat(predicted + targets, dim=1)]
    return X

valid_dl = torch.utils.data.DataLoader(dataset=valid_ds, 
                                       collate_fn=valid_ds.collate_fn,
                                       num_workers=0,
                                       batch_size=128)

In [41]:
preds = torch.vstack(inference(model_multihead, valid_dl)).cpu().numpy()

100%|██████████| 3358/3358 [00:49<00:00, 68.45it/s]


In [42]:
import numpy as np

df_valid = pd.DataFrame(preds, columns = ['predicted_mcc', 'predicted_amnt', 'predicted_hour_diff', 'mcc', 'amnt', 'hour_diff'])
df_valid.head()

,predicted_mcc,predicted_amnt,predicted_hour_diff,mcc,amnt,hour_diff
0,2.0,0.409407,0.005357,9.0,0.352101,0.000000
1,1.0,0.310597,0.001620,2.0,0.539584,0.006944
2,61.0,0.288133,0.001551,61.0,0.232139,0.000000
3,9.0,0.332158,0.005253,1.0,0.352706,0.133333
4,5.0,0.334019,0.000990,2.0,0.477476,0.358333


In [50]:
df_valid['denormalized'] = df_valid['predicted_hour_diff'] * (max_value - min_value) + min_value
df_valid['denormalized_2'] = df_valid['hour_diff'] * (max_value - min_value) + min_value

In [51]:
df_valid

,predicted_mcc,predicted_amnt,predicted_hour_diff,mcc,amnt,hour_diff,denormalized,denormalized_2
0,2.0,0.409407,0.005357,9.0,0.352101,0.000000,3.856905,0.000000
1,1.0,0.310597,0.001620,2.0,0.539584,0.006944,1.166646,5.000000
2,61.0,0.288133,0.001551,61.0,0.232139,0.000000,1.116806,0.000000
3,9.0,0.332158,0.005253,1.0,0.352706,0.133333,3.781850,96.000008
4,5.0,0.334019,0.000990,2.0,0.477476,0.358333,0.712511,258.000000
...,...,...,...,...,...,...,...,...
429706,1.0,0.299527,0.000690,1.0,0.299375,0.011111,0.496812,8.000000
429707,3.0,0.355704,0.000452,16.0,0.420604,0.061111,0.325509,44.000000
429708,18.0,0.413712,0.005822,18.0,0.229750,0.006944,4.191970,5.000000
429709,1.0,0.363661,0.000499,1.0,0.320061,0.009722,0.359510,7.000000


In [52]:
df

,app_id,amnt,mcc,hour_diff,normalized
0,0,"[0.3363445046218684, 0.3590500655756334, 0.383...","[2, 20, 9, 2, 2, 9, 2, 2, 2, 63, 2, 2, 9, 2, 9...","[0, 74, 0, 23, 0, 47, 3, 139, 79, 44, 75, 0, 1...","[0.0, 0.10277777777777777, 0.0, 0.031944444444..."
1,1,"[0.43906011449663346, 0.3233134807595583, 0.34...","[2, 1, 2, 3, 2, 1, 1, 1, 1, 1, 1, 2, 1, 58, 1,...","[37, 0, 30, 74, 42, 0, 18, 82, 14, 24, 23, 8, ...","[0.05138888888888889, 0.0, 0.04166666666666666..."
2,2,"[0.3208046754153104, 0.4852952089625992, 0.389...","[28, 2, 9, 9, 9, 1, 1, 61, 9, 1, 61, 61, 9, 1,...","[156, 10, 378, 1, 1, 0, 0, 42, 52, 0, 17, 24, ...","[0.21666666666666667, 0.013888888888888888, 0...."
3,4,"[0.3876771200456198, 0.39302024879044656, 0.39...","[9, 9, 2, 9, 9, 9, 20, 1, 1, 2, 1]","[73, 0, 19, 162, 0, 158, 0, 44, 20, 0, 96]","[0.10138888888888889, 0.0, 0.02638888888888889..."
4,6,"[0.41144777426121326, 0.38480180399505215, 0.3...","[58, 8, 3, 3, 1, 14, 14, 2, 2, 2, 2, 14, 14, 5...","[24, 1, 46, 25, 1, 115, 0, 1, 25, 0, 20, 55, 0...","[0.03333333333333333, 0.001388888888888889, 0...."
...,...,...,...,...,...
429706,1003041,"[0.28153909926329146, 0.21628504289587916, 0.2...","[3, 1, 1, 3, 1, 1, 93, 1, 1, 4, 1, 3, 81, 2, 1...","[17, 109, 24, 35, 149, 7, 112, 0, 19, 5, 24, 2...","[0.02361111111111111, 0.15138888888888888, 0.0..."
429707,1003044,"[0.33499195590768144, 0.33347360274761184, 0.2...","[4, 3, 4, 24, 24, 4, 19, 4, 2, 3, 5, 5, 72, 5,...","[0, 21, 27, 25, 0, 0, 21, 0, 19, 4, 4, 0, 22, ...","[0.0, 0.029166666666666667, 0.0375, 0.03472222..."
429708,1003047,"[0.5162697499113479, 0.49816793314835256, 0.54...","[15, 15, 15, 4, 4, 4, 1, 1, 1, 1, 11, 18, 18, ...","[42, 0, 52, 4, 0, 0, 88, 0, 30, 0, 18, 226, 0,...","[0.058333333333333334, 0.0, 0.0722222222222222..."
429709,1003048,"[0.5894817634813461, 0.5836670249268464, 0.479...","[12, 2, 12, 12, 1, 1, 12, 1, 1, 84, 3, 7, 1, 3...","[143, 0, 24, 2, 20, 10, 1, 23, 23, 16, 3, 1, 1...","[0.1986111111111111, 0.0, 0.03333333333333333,..."


# <font size="5">Metrics</font>

In [29]:
from sklearn.metrics import accuracy_score

print("Accuracy:", {accuracy_score(df_valid['mcc'],  df_valid['predicted_mcc'])})

Accuracy: {0.3547942687061769}


In [30]:
from sklearn.metrics import mean_absolute_error

print("Mae amnt:", {mean_absolute_error(df_valid['amnt'],  df_valid['predicted_amnt'])})

Mae amnt: {0.07482444}


In [37]:
from sklearn.metrics import mean_absolute_error

print("MAE:", {mean_absolute_error(df_valid['denormalized_2'],  df_valid['denormalized'])})

MAE: {43.338947}
